In [34]:
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable,lpDot
import numpy as np


def soma_ponderada(n,A,b,C,d,coef_func,vetor_lambda):
    model = LpProblem(name="large-problem", sense=LpMaximize)
   # Neste trabalho será tratado apenas dos casos lineares com variáveis reais

    x = [LpVariable(name=f"x{i}", lowBound=0) for i in range(n)]
    # Restrições <=
    if isinstance(A, (list, np.ndarray)) and isinstance(b, (list, np.ndarray)):
        rest1 = A.shape[0]
    
    # Criamos Uma lista que contêm todas as variáveis do problema.

        for i in range(rest1):
            model += (lpDot(A[i], x) <= b[i], f"restrição {i}")
    else:
        pass
    if isinstance(A, (list, np.ndarray)) and isinstance(b, (list, np.ndarray)):
        rest2 = C.shape[0]
        
        # Restrições >=
        
        for i in range(rest2):
            model += (lpDot(C[i], x) >= d[i], f"restrição {i + rest1}")
    else:
        pass

    # Funções Objetivas
    

    funcao_principal =  np.transpose(vetor_lambda)@coef_func@x
    model += funcao_principal
    f_values = coef_func@x


    

    status = model.solve()
    sol = np.zeros(n)
    for i in range(n):
        sol[i] = x[i].value()
    print(f"Solução: {sol}")
    sol = np.transpose(sol)
    value_func_aux = np.zeros(coef_func.shape[0])
    for i in range(coef_func.shape[0]):
     value_func_aux[i] = np.dot(sol,coef_func[i, :])
    for i in range(coef_func.shape[0]):
        print(f"O valor de f{i+2} é: {value_func_aux[i]}")



    print(f"status: {model.status}, {LpStatus[model.status]}")
    #print(f"Objetivo: {model.objective.value()}")
   # print(model)

In [20]:
n=2
A = np.array([[1,3]])
b = np.array([3])
C = np.array([[1,1]])
d=np.array([1])
coef_func=np.array([[1,1],[2,1]])
vetor_lambda = np.array([1,2])

soma_ponderada(n,A,b,C,d,coef_func,vetor_lambda)

Solução: [3. 0.]
O valor de f2 é: 3.0
O valor de f3 é: 6.0
status: 1, Optimal
Objetivo: 15.0
large-problem:
MAXIMIZE
5*x0 + 3*x1 + 0
SUBJECT TO
restrição_0: x0 + 3 x1 <= 3

restrição_1: x0 + x1 >= 1

VARIABLES
x0 Continuous
x1 Continuous



In [40]:
"""
n=6
A = np.array([[216,164,456,516,176,670],[28.1,13.6,0,0,5,0],[1.6,8.5,0,0,0,0],[166.1,167.1,205.1,209.1,236.1,228.1],[2.05,2.1,8.35,11.95,8.2,13.8]])
d=np.array([1800,100,25,1300,22])
C = A
#d=np.array([100,22,1500,25,1800])
b = np.array([2500,150,96,2300,29])
coef_func=np.array([[-0.8,-0.9,-2.7,-4,-6.5,-2.8],[2.5,4.8,32,32.4,14.6,13.8]])
"""
n=6
A = np.array([[216,164,456,516,176,670],[28,14,0,0,5,0],[2,8,0,0,0,0],[166,167,205,209,236,228],[2,2,8,12,8,13]])
d=np.array([1800,100,25,1300,22])
C = A
#d=np.array([100,22,1500,25,1800])
b = np.array([2500,150,36,2300,29])
coef_func=np.array([[-0.8,-0.9,-2.7,-4,-6.5,-2.8],[3,5,32,32,15,14]])
vetor_lambda = np.array([36,1])
for i in range(36):
    print(f'Os valores das funções, com lb1 = {i+1} e lb2 = 1 é: ')
    soma_ponderada(n,A,b,C,d,coef_func,np.array([i+1,1]))

Os valores das funções, com lb1 = 1 e lb2 = 1 é: 
Solução: [2.2959184 2.5510204 2.4132653 0.        0.        0.       ]
O valor de f2 é: -10.648469389999999
O valor de f3 é: 96.8673468
status: 1, Optimal
Os valores das funções, com lb1 = 2 e lb2 = 1 é: 
Solução: [2.2959184 2.5510204 2.4132653 0.        0.        0.       ]
O valor de f2 é: -10.648469389999999
O valor de f3 é: 96.8673468
status: 1, Optimal
Os valores das funções, com lb1 = 3 e lb2 = 1 é: 


Solução: [2.2959184 2.5510204 2.4132653 0.        0.        0.       ]
O valor de f2 é: -10.648469389999999
O valor de f3 é: 96.8673468
status: 1, Optimal
Os valores das funções, com lb1 = 4 e lb2 = 1 é: 
Solução: [2.2959184 2.5510204 2.4132653 0.        0.        0.       ]
O valor de f2 é: -10.648469389999999
O valor de f3 é: 96.8673468
status: 1, Optimal
Os valores das funções, com lb1 = 5 e lb2 = 1 é: 
Solução: [2.2959184 2.5510204 2.4132653 0.        0.        0.       ]
O valor de f2 é: -10.648469389999999
O valor de f3 é: 96.8673468
status: 1, Optimal
Os valores das funções, com lb1 = 6 e lb2 = 1 é: 
Solução: [2.2959184 2.5510204 2.4132653 0.        0.        0.       ]
O valor de f2 é: -10.648469389999999
O valor de f3 é: 96.8673468
status: 1, Optimal
Os valores das funções, com lb1 = 7 e lb2 = 1 é: 
Solução: [2.2959184 2.5510204 2.4132653 0.        0.        0.       ]
O valor de f2 é: -10.648469389999999
O valor de f3 é: 96.8673468
status: 1, Optimal
Os valores das funções, c

In [18]:
aaa = np.array([[1,2],[3,4],[5,6]])
aaa.shape[1]

2